Dokumentace k projektu "Live Survey"
=================================================== 

**Live Survey** je služba pro tvorbu a správu anket s možností několika typů odpovědí. 

Tento projekt je inspirován aplikací Mentimeter.
Jako úložiště dat je používán databázový systém PostgreSQL. Frontend je realizovaný v Reactu, backend je psaný v Pythonu.

#### Stručný popis
Projekt se skládá z webového rozhraní pro uživatele, které je vytvořeno pomocí Reactu a Bootstrapu. Web je koncipován do prohlížecí a vytvářecí části (vytvářecí část je v procesu tvorby). Uživatel si může prohlížet ankety uložené lokálně v adresáři jako JSONdata. Pomocí přehledu (/ui/album) si zvolí zobrazené ankety, které jsou v jednotlivých kartách jako náhledy. Po přesměrování na konkrétní anketu uživatel odpoví na zadané otázky podle jejich typu (volná odpověď - text, výběr více možností - checkbox, výběr jedné z více možností - radio button). Po zmáčknutí tlačítka "odeslat" dojde k odeslání dat na API adresu /api/sendAsnw.

Pro příjem dat slouží FastAPI, která má v aktuální fázi našeho projektu pouze triviální metody. Následně je v plánu uložení dat na Postgres místo lokální databáze.

Frontend, FastAPI a Postgres jsou zapouzdřeny v NGINX pro vzájemnou komunikaci. 

Splněné podmínky:
========================================================

- Projekt je uložený na public úložišti [Github](https://github.com/EditaBozkova/LiveSurvey)
- WEB Root UI je konfigurovatelný (URI)
- Klíčové funkcionality jsou popsány a demonstrovány pomocí notebooku v Jupyteru
- Je definovaný dockerfile / compose.yml pro spuštění výsledného projektu v prostředí docker
- [Docker images](https://hub.docker.com/r/pompino/live_survey) jsou publikovány na hub.docker.com
- Je nastavena otevřená licence (MIT)

To do list:
===============================

- Propojení s Postgress (aktuálně je propojen pouze s lokální databází)
- Funkčí odstranění anket
- Tvorba kompletních anket
- Správná metoda POST pro posílání odpovědí

Spuštění aplikace:
=========================

- Zajištění dat z repositáře na [GitHub](https://github.com/EditaBozkova/LiveSurvey)
- Podmínka pro nasazení projektu je mít nainstalovaný Docker Hub
- Pro sestavení kontejneru je zapotřebí spustit soubor runCompose.bat (provede automatické Compose up ale i Compose down s odstraněním)
- Spustíme webové rozhraní UI přes NGINX ve webovém prohlížeči (pro vzájemnou komunikaci s FastAPI)

Úpravy před spuštěním
==========================

#### Data vytvořených anket - struktura JSON
Pro přidání nových anket, je zapotřebí úpravy zdrojového souboru AlbumData.js, který obsahuje jedinou proměnou ve formátu JSON. Tento soubor je strukturován jednotlivých anket a každá anketa je rozdělena na `info` a `questions`. 

`info` obsahuje:
- SName: Zobrazovaný název ankety
- ID: Identifikátor ankety
- SAutor: Zobrazovaný autor ankety
- SAbout: Stručný popis ankety
- STime: Přibližná doba splnění ankety

`questions` obsahuje pole jednotlivých otázek a ty obsahují:
- Type: Typ otázky (text, radio, checkbox)
- ID: Identifikátor otázky
- QTitle: Znění otázky
- Choices: Pole možností odpovědí (pouze u radio a checkbox)

#### Konfigurování WEB Root
Pro konfiguraci WEB root UI nám slouží nging.conf, který obsahuje údaje pro konfiguraci UI i API. UI funguje na adrese /ui a FastAPI na /api

Popis programů
=====================

## `frontend` : 

Frontend je Reactová aplikace, webové rozhraní pro uživatele. Adresa domovské stránky se nachází na adrese /ui/, která obsahuje horní navigační lištu (tento NavigationBar obsahují všechny stránky), která obsahuje pár odkazů (na repositář projektu, profily autorů a jiné odkazy na webu). Aplikace je tvořena style single page, tudíž díky `React-router-dom`, který se nachází v app.js, přesměrovává pomocí URL adresy.

Z domovské stránky se lze dostat na vytváření nové ankety a nebo na prohlížení již hotvých anket.

#### Tvorba anket:
Vytváření nových anket je zatím ve fázi vytvoření hlavičky pro anketu (Název a autor)

Data od uživatele jsou získána pomocí komponent `input` - text
Odeslaná data jsou ve formátu {autor: str, nazev: str}

#### Album anket:
Po přesměrování na prohlížení anket, se dynamicky zobrazí webová stránka, podle počtu existujích anket. Jednotlivé náhledy anket se zobrazují v komponentách `Card`.

Náhled obsahuje základní informace jako název, autora, počet otázek a odkaz pro spuštění konkrétní ankety. Dále obsahuje i tlačítko pro smazání konrétní ankety, ale to v této verzi nezastává ještě konrétní funkcionalitu.

#### Konkrétní anketa:

K přesměrování ke konrétní anketě dojde z alba, kdy URL obsahuje `/ui/album/:id/:nameid`, kde id je index konrétní ankety a nameid je identifikátor ankety.

Stránka je opětovně dynamicky generována jako předchozí (Album) pomocí jednotlivých karet, které obsahují již konkrétní otázky. Každá karta obsahuje svoji otázku, pořadí otázky a způsob zadání odpovědi. 

Otázky jsou buď typu text: Způsob zadání je tedy jeden `input` type:text

Otázky typu radio: Způsob zadání je tedy dynamické množství `input` type:radio, kdy lze vybrat pouze jednu odpověď z jedné otázky

Otázky typu checkbox: Způsob zadání je tedy dynamické množství `input` type:checkbox, kdy lze vybrat libovolný počet odpovědí z jedné otázky

Na konci dotazníku je tlačítko Odeslat (`input` type:submit), který odešle odpovědi na adresu /api/sendAsnw ve formátu [{`id_otazky`:`odpoved`}].

## `fastAPI` : 

Backend je napsaný pomocí Pythonu a frameworku FastAPI. Pro dokumentaci FastAPI používáme integoravé postředí Swagger UI, které ve webovém prohlížeči prezentuje všechny vytvořené metody (POST, GET, Delete) a lze je jednoduše vyzkoušet.

Náš projekt obsahuje metody pro zpracování dat z vytvoření ankety (pouze název a autor). Pro tuto funkcionalitu máme metodu POST, kterou využívá komponenta CreateS.js. Dále je zde metoda GET a Delete, která ve frontendu není nikterak použita. 

Dále obsahuje metody pro zpracování odeslaných odpovědí, které v aktuální verzi postrádají kýženou funkcionalitu.

Doporučené odkazy
=================

[Vytvoření React aplikace](https://reactjs.org/docs/create-a-new-react-app.html)

[Komponenty pro tvorbu webu React-bootstrap](https://react-bootstrap.github.io)

[Dokumentace k FastAPI](https://fastapi.tiangolo.com)

Autoři
=================

- des. Daniel Popelář ([Pompino](https://github.com/Pompino))
- des. Edita Božková ([EditaBozkova](https://github.com/EditaBozkova))